In [8]:
import pandas as pd

In [9]:
df_shelter_effect = pd.read_excel("https://www.insee.fr/fr/statistiques/fichier/6524161/ia75.xlsx",
                                  sheet_name="Figure 3", skiprows=3, skipfooter=7)

In [10]:

df_shelter_effect = df_shelter_effect.rename(columns={'Unnamed: 0': 'long_coicop',
                                                      'Effet total.1': 'inflation_with_shelter', 'Effet total': 'inflation_without_shelter'})
df_shelter_effect = df_shelter_effect.set_index('long_coicop')
df_shelter_effect = df_shelter_effect.drop(["  électricité, gaz et autres combustibles",
                                            "  dépenses d'utilisation de véhicules"])

In [11]:
df_shelter_effect['shelter_effect'] = df_shelter_effect['inflation_with_shelter'] - \
    df_shelter_effect['inflation_without_shelter']

In [12]:

df_shelter_effect = df_shelter_effect.drop(['Unnamed: 1', 'Effet direct', 'Effet indirect',
                                            'Effet direct.1', 'Effet indirect.1'], axis=1)

In [13]:
# Create a list of values for the "coicop" column
coicop_values = ['{:02d}'.format(i)
                 for i in range(1, len(df_shelter_effect)+1)]

# Add the "coicop" column to the DataFrame
df_shelter_effect['coicop'] = coicop_values

In [14]:
df_shelter_effect = df_shelter_effect.set_index('coicop')

In [15]:

df_shelter_effect = df_shelter_effect.drop(
    ["inflation_without_shelter", "inflation_with_shelter"], axis=1)

In [16]:
import pandas as pd


url = "https://www.insee.fr/fr/statistiques/fichier/4648335/TF106.csv"

df_prop = pd.read_csv(url, sep=";")

discriminatingColumn = "DECUC"

In [17]:
# filtering values with exactly 2 characters, corresponding to ECOICOP classes
df_prop = df_prop.loc[df_prop['NOMENCLATURE'].str.len() == 2]

In [18]:
# renaming the "NOMENCLATURE" column to "coicop"
df_prop = df_prop.rename(columns={'NOMENCLATURE': 'coicop'})

In [19]:
# filtering out rows where the "coicop" value starts with "13"
df_prop = df_prop.loc[~df_prop['coicop'].str.startswith('13')]

In [20]:
# calculating the sum of "CONSO" for each unique value in "DECUC" column
sum_by_decuc = df_prop.groupby(discriminatingColumn)['CONSO'].sum()

In [21]:
# adding a new column "PROP" with the percentage of each row relative to the sum for each unique value in "DECUC" column
df_prop['PROP'] = df_prop['CONSO'] / \
    df_prop[discriminatingColumn].map(sum_by_decuc)

In [23]:
df_prop = df_prop.merge(right=df_shelter_effect, on="coicop")

In [24]:
df_prop["inflation_avoided"] = df_prop["PROP"]*df_prop["shelter_effect"]

In [25]:
df_prop.groupby(discriminatingColumn)['inflation_avoided'].sum()

DECUC
1      2.918762
10     2.509418
2      2.884493
3      2.843787
4      2.804665
5      2.763964
6      2.718840
7      2.656306
8      2.576934
9      2.502113
TOT    2.672702
Name: inflation_avoided, dtype: float64